## 🥉 Camada Bronze – Coleta e Validações


#### 📌 Passo 1: Acessando o Storage da Azure 

###### Por motivos de problemas com o Unity Catalog, depois de pesquisas achei a alternativa de Montar com "dbutils" e até o momento está funcional

In [0]:
# Acesso do Blob Storage
container_name = "lake"
storage_account_name = "lakesavana"
mount_point = "/mnt/savana"
access_key = "M4OYEhV7y/mOowqVqZRcLfm2taCeU7eF8WEROHP5gEV22EbpDFUZx5OsCtzjeoyJ2fna0FRGjqaz+AStYvlGeA=="

# Configurar acesso
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    access_key
)


#### 📌 Passo 2: Leitura dos Arquivos "Parquet"

###### Agora vou Listar o que tem Dentro do Storage, ler o arquivo como Dataframe para ver como estão os vindo os Dados

In [0]:
# Listar os Arquivos do Blob (Finalidade é para confirmar que está funcionando a Conexão)
dbutils.fs.ls("/mnt/savana/__unitystorage/catalogs/10af6d1d-32a1-46c7-95d7-ef4fbde791d4/tables/")


In [0]:
# Lê cada pasta
df_transacoes = spark.read.format("delta").load("/mnt/savana/__unitystorage/catalogs/10af6d1d-32a1-46c7-95d7-ef4fbde791d4/tables/0a6c3f04-3748-4d9e-9b2f-6400c0dc680b")
df_clientes = spark.read.format("delta").load("/mnt/savana/__unitystorage/catalogs/10af6d1d-32a1-46c7-95d7-ef4fbde791d4/tables/290d3ddb-6031-495f-a097-fbb09f50f4d8")

# Exibir os dados
df_transacoes.display()
df_clientes.display()


#### 📌 Passo 3: Efetuar a Camada Bronze

###### Agora irei pegar as Duas tabelas (Base de Clientes / Transações) que transformei em Dataframe e jogar para dentro do meu banco

In [0]:
# Tabelas Bronze no catálogo
df_transacoes.write.mode("overwrite").saveAsTable("adb_cliente_savana_prd.daniel_bronze.transacoes_teste")
df_clientes.write.mode("overwrite").saveAsTable("adb_cliente_savana_prd.daniel_bronze.clientes_teste")


#### 📌 Passo 4: Verificar Conteudo das Colunas 
###### Antes de Entrar na "Camada Silver" e Realizar os Tratamentos fui em buscar de saber o que tem dentro das colunas para ter uma visão do que tratar

In [0]:
from pyspark.sql.functions import col, count, trim, regexp_extract

# Analisar os valores únicos da coluna Nome (Base de Cliente)
df_clientes.groupBy(trim(col("Nome")).alias("Nome")) \
  .agg(count("*").alias("Quantidade")) \
  .orderBy("Quantidade", ascending=False) \
  .display()

# Analisar os valores únicos da coluna Cidade (Base de Clientes)
df_clientes.groupBy(trim(col("Cidade")).alias("Cidade")) \
  .agg(count("*").alias("Quantidade")) \
  .orderBy("Quantidade", ascending=False) \
  .display()

# Analisar os valores únicos da coluna CPF (Base de Clientes)
df_clientes.groupBy(trim(col("CPF")).alias("CPF")) \
  .agg(count("*").alias("Quantidade")) \
  .orderBy("Quantidade", ascending=False) \
  .display()

# Verificar se tem alguma string dentro da coluna CPF (Base de Clientes)
df_clientes.withColumn("CPF", regexp_extract(col("CPF"), r'[a-zA-Z]', 0)) \
  .filter(col("CPF") != "") \
  .groupBy(trim(col("CPF")).alias("CPF")) \
  .agg(count("*").alias("Quantidade")) \
  .orderBy("Quantidade", ascending=False) \
  .display()

# -----------------------------------------------------------------------------------------------

# Analisar os valores únicos da coluna Meio De Pagamento (Transações)
df_transacoes.groupBy(trim(col("Meio_de_Pgmto")).alias("Meio_de_Pgmto")) \
  .agg(count("*").alias("Quantidade")) \
  .orderBy("Quantidade", ascending=False) \
  .display()

# Analisar os valores únicos da coluna Tipo (Transações)
df_transacoes.groupBy(trim(col("Tipo")).alias("Tipo")) \
  .agg(count("*").alias("Quantidade")) \
  .orderBy("Quantidade", ascending=False) \
  .display()
